In [1]:
import re
import numpy as np
import cupy as cp
from pyqcu.cuda import define
from pyqcu.cuda import io
from pyqcu.cuda import qcu
from pyqcu.cuda.set import params, argv, set_ptrs
import h5py
print('My rank is ', define.rank)
gauge_filename = f"quda_wilson-bistabcg-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.h5"
params[define._GRID_T_] = 1
params[define._NODE_RANK_] = define.rank
params[define._NODE_SIZE_] = define.size
params[define._DATA_TYPE_] = define._LAT_C64_
params[define._SET_PLAN_] = 1
params[define._VERBOSE_] = 1
argv[define._MASS_] = 0.0
print("Parameters:", params)
#############################
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_out_filename)
#############################
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
#############################




    @@@@@@######QCU NOTES START######@@@@@@@
    Guide:
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (wilson:x=4,y=4,z=4,t=4;clover:x=8,y=8,z=8,t=8) that QCU support (when '#define _B

In [2]:

qcu.applyInitQcu(set_ptrs, params, argv)
qcu.applyWilsonBistabCgQcu(
    fermion_out, fermion_in, gauge, set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)
#############################
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))
#############################
io.grid_xxxtzyx2hdf5_xxxtzyx(fermion_out, params)


set_ptr:0x5608c2234730
long long set_ptr:94595616818992
gridDim.x               :32768
blockDim.x              :16
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_DATA_TYPE_]:3
host_params[_SET_INDEX_]:0
host_params[_SET_PLAN_] :1
host_params[_MG_X_]     :4
host_params[_MG_Y_]     :4
host_params[_MG_Z_]     :4
host_params[_MG_T_]     :8
host_params[_LAT_E_]    :24
host_params[_VERBOSE_]  :1
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :10

Difference: 3.056118e-07
Input Array Shape: (2, 4, 3, 32, 32, 32, 16)
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Rank 0: Data is saved to xxxtzyx.h5


In [17]:
import torch
from pyqcu.ascend import dslash_parity
from pyqcu.ascend import inverse
dof = 12
latt_size = (8, 8, 8, 8)
# latt_size = (4, 4, 4, 4)
kappa = 0.125
# dtype = torch.complex128
dtype = torch.complex64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# Initialize lattice gauge theory
wilson = dslash_parity.wilson_parity(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
clover = dslash_parity.clover_parity(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
U = wilson.generate_gauge_field(sigma=0.1, seed=42)
null_vecs = torch.randn(dof, 4, 3, latt_size[3], latt_size[2], latt_size[1], latt_size[0],
                        dtype=dtype, device=device)
clover_term = clover.make_clover(U=U)


def matvec(src: torch.Tensor, U: torch.Tensor = U) -> torch.Tensor:
    return wilson.give_wilson(src, U)+clover.give_clover(clover=clover_term, src=src)
    return wilson.give_wilson(src, U)


# 生成近似零空间向量
null_vecs = inverse.give_null_vecs(
    null_vecs=null_vecs,
    matvec=matvec,
    verbose=False
)

Using device: cuda


In [20]:
for i in range(dof):
    print(f"A*v/v check again:")
    Av = matvec(null_vecs[i])
    print(f"  Vector {i}: ||A*v|| = {torch.norm(Av).item():.6e}")
    # print(
    #     f"  Vector {i}: v = {null_vecs[i]}")
    # print(
    #     f"  Vector {i}: A*v = {Av}")
    # print(
    #     f"  Vector {i}: A*v/v = {Av/null_vecs[i]}")
    print(
        f"torch.norm(null_vecs[{i}]).item():.6e:{torch.norm(null_vecs[i]).item():.6e}")
    # orthogonalization
    for k in range(0, i+1):
        print(
            f"torch.vdot(null_vecs[{i}].flatten(), null_vecs[{k}].flatten()):{torch.vdot(null_vecs[i].flatten(), null_vecs[k].flatten())}")

A*v/v check again:
  Vector 0: ||A*v|| = 3.055307e-02
torch.norm(null_vecs[0]).item():.6e:1.000000e+00
torch.vdot(null_vecs[0].flatten(), null_vecs[0].flatten()):(1-1.3839012401317685e-11j)
A*v/v check again:
  Vector 1: ||A*v|| = 2.612895e-02
torch.norm(null_vecs[1]).item():.6e:1.000000e+00
torch.vdot(null_vecs[1].flatten(), null_vecs[0].flatten()):(-3.015156835317612e-08-0.4090597927570343j)
torch.vdot(null_vecs[1].flatten(), null_vecs[1].flatten()):(1.0000001192092896-5.3160018054221325e-11j)
A*v/v check again:
  Vector 2: ||A*v|| = 2.577162e-02
torch.norm(null_vecs[2]).item():.6e:1.000000e+00
torch.vdot(null_vecs[2].flatten(), null_vecs[0].flatten()):(0.07234969735145569+0.6980615258216858j)
torch.vdot(null_vecs[2].flatten(), null_vecs[1].flatten()):(-1.4901161193847656e-08+0.35373663902282715j)
torch.vdot(null_vecs[2].flatten(), null_vecs[2].flatten()):(1.0000001192092896+3.525450070762659e-11j)
A*v/v check again:
  Vector 3: ||A*v|| = 3.558065e-02
torch.norm(null_vecs[3]).item():

# vdot()=dot(conj(),)